In [1]:
import pandas as pd
import time
import random
import requests
from bs4 import BeautifulSoup
import re
import json
from tqdm import tqdm
from unidecode import unidecode 

### Funciones básicas para realizar scraping
#### Funciones para limpiar texto y moneda

In [2]:
#Función para obtener tipo de cambio
def usd():
    #Obtener tipo de cambio
    banxico="https://www.banxico.org.mx/SieAPIRest/service/v1/series/SF43718/datos/?token=0e825df61e5eca2dd60340f1d39766f5cbefc052fb00f49b257095da3e004921"
    r=requests.get(banxico).json()
    #Obtener último dato
    mxn=r["bmx"]["series"][0]["datos"][-1]["dato"]
    #transformar a float
    mxn=float(mxn)
    return mxn

In [3]:
def limpia_texto(text):
    if text is None:
        return ""
    # Elimina caracteres no alfanuméricos, caracteres, puntuación, espacios extras y signos de pesos
    cleaned_text = re.sub(r'[^\w\s.]', '', text).strip()
    # Minúsculas
    cleaned_text = cleaned_text.lower()
    #Eliminar acentos
    cleaned_text = unidecode(cleaned_text)
    return cleaned_text

def limpia_moneda(text):
    if text is None:
        return ""
    #Eliminar "\n"
    cleaned_coin = re.sub(r'\n', '', text).strip()
    #Elimina comas
    cleaned_coin = re.sub(r',', '', text).strip()
    #Eliminar signo de pesos
    cleaned_coin = re.sub(r'$', '', cleaned_coin)

    return cleaned_coin

In [4]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}



def equalize_lists(main_list, *lists):
    main_length = len(main_list)
    for lst in lists:
        while len(lst) < main_length:
            lst.append(None)

In [5]:
def easybroker(municipio, estado,tipo="venta"):
    if tipo == "venta":
        base_url = "https://www.easyaviso.com/mx/inmuebles/propiedades-residenciales-en-venta-en-{}-{}?page={}"
    elif tipo == "renta":
        base_url = "https://www.easyaviso.com/mx/inmuebles/propiedades-residenciales-en-renta-en-{}-{}?page={}"
    elif tipo == "terreno":
        base_url = "https://www.easyaviso.com/mx/inmuebles/terrenos-en-venta-en-{}-{}?page={}"
    else:
        raise ValueError("Selecciona un tipo de propiedad válido: venta, renta o terreno")

    all_data_frames = []

    for page_num in tqdm(range(1, 54), desc=f"Scrapeando Easybroker en {municipio}"):
        url = base_url.format(municipio,estado,page_num)
        r = requests.get(url, headers=headers)
        sopa = BeautifulSoup(r.text, 'html.parser')

        # Cosas a obtener
        recamaras, bathrooms, superficie, direcciones, ofertas, precios, latitud, longitud = [], [], [], [], [], [], [], []

        for precio in sopa.find_all('li', class_='price'):
            precios.append(precio.text.strip())
        for coord in sopa.find_all('li', class_='property__component'):
            latitud.append(coord.get('data-lat'))
            longitud.append(coord.get('data-long'))
        for div in sopa.find_all('div', class_='features'):
            match = re.search(r'(\d+)\s*recámaras', div.text)
            recamaras.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+)\s*baños', div.text)
            bathrooms.append(int(match.group(1)) if match else None)
            match = re.search(r'(\d+\.?\d*)\s*m²', div.text)
            superficie.append(float(match.group(1)) if match else None)
        for element in sopa.find_all('div', class_='property__content property-content'):
            direcciones.append(element.find('div', class_='location').text.strip())
        for title in sopa.find_all('a', class_='title'):
            ofertas.append(title.text.strip())

                # Filtrar precios según el tipo de propiedad
        if tipo in ["venta", "terreno"]:
            precios = [price for price in precios if "En Renta" not in price]
        elif tipo == "renta":
            precios = [price for price in precios if "En Venta" not in price]

        equalize_lists(ofertas, recamaras, bathrooms, superficie, direcciones, precios, latitud, longitud)

        data_frame = pd.DataFrame({
            'oferta': ofertas, 'precio': precios, 'mts': superficie,
            'bathrooms': bathrooms, 'recamaras': recamaras, 'lat': latitud,
            'lon': longitud, 'fuente': 'easybroker'
        })
        all_data_frames.append(data_frame)
    combined_df = pd.concat(all_data_frames, ignore_index=True)
    
    if combined_df.empty:
        return combined_df
    
    if tipo in ["venta", "terreno"]:
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Venta", "", regex=False)
                                .str.replace("\n","", regex=False))
    elif tipo == "renta":
        combined_df["precio"] =(combined_df["precio"].apply(limpia_moneda)
                                .str.replace("$", "", regex=False)
                                .str.replace("Consulte precio", "0", regex=False)
                                .str.replace("En Renta", "", regex=False)
                                .str.replace("\n","", regex=False))
        
    combined_df = combined_df[~combined_df["precio"].str.contains("por ha")]
    combined_df["precio"] = combined_df["precio"].apply(lambda x: float(x.replace("US", "")) * usd() if "US" in x else x)
    combined_df["precio"] = pd.to_numeric(combined_df["precio"], errors="coerce")
    #Eliminar nans
    combined_df = combined_df[combined_df["precio"].notna()]
        #Precio a float
    combined_df["precio"] = combined_df["precio"].astype(float)
        #Eliminar "\n" de precio
    combined_df = combined_df[combined_df["precio"] != 0]
    # Añadir fecha de consulta
    combined_df["fecha_consulta"] = pd.to_datetime("today")
    #Añadir fuente
    combined_df["fuente"] = "easybroker"
    #Añadir municipio
    combined_df["municipio"] = municipio
    #Limpiar oferta
    combined_df["oferta"] = combined_df["oferta"].apply(limpia_texto)
    return combined_df

In [6]:
def limpia_datos(df):
    df = df.reset_index(drop=True)
   
    #Eliminar registros con precio 0 o nan
    df=df[df['precio']>0]
    df=df[df['precio'].notna()]
    #Eliminar registros que en oferta contengan "terreno"
    df=df[~df['oferta'].str.contains('terreno')]
    df=df[~df['oferta'].str.contains('remodelar')]
    df=df[~df['oferta'].str.contains('hectareas')]
    #Si la fuente es goodlers, sacar el promedio de precio_min y precio_max y ponerlo en precio
    #Eliminar registros con misma oferta y mismo precio
    df=df.drop_duplicates(subset=['oferta','precio','recamaras','bathrooms'],keep='first')
    #Calcular precio por metro cuadrado
    df['precio_m2'] = df['precio'] / df['mts']

    return df

#### Realizar webscraping para viviendas en venta

In [7]:
#Scrapear lamudi
viviendas=easybroker("mazatlan","sinaloa","venta")

Scrapeando Easybroker en mazatlan: 100%|██████████| 1/1 [00:00<00:00,  1.11it/s]


In [8]:
vivi_limpia=viviendas.copy()
#Eliminar si oferta dice "lote" o "terreno"
vivi_limpia=vivi_limpia[~vivi_limpia["oferta"].str.contains("lote|terreno")]
#Eliminar si lat es nulo
vivi_limpia=vivi_limpia[vivi_limpia['lat'].notna()]
#Aplicar función de limpieza
vivi_limpia=limpia_datos(vivi_limpia)
vivi_limpia

,oferta,precio,mts,bathrooms,recamaras,lat,lon,fuente,fecha_consulta,municipio,precio_m2
0,casa en venta soles residencial 1871 mazatlan ...,7200000.00,236.00,4.0,4.0,23.2626504,-106.4627078,easybroker,2024-09-02 12:14:20.690010,mazatlan,30508.474576
1,departamento en venta en torre isla balboa,3820000.00,87.48,2.0,2.0,23.251008,-106.4520348,easybroker,2024-09-02 12:14:20.690010,mazatlan,43667.123914
2,casa en venta jacarandas mazatlan sinaloa,3400000.00,149.00,2.0,3.0,23.2433991,-106.4180536,easybroker,2024-09-02 12:14:20.690010,mazatlan,22818.791946
3,nautilus marina gran vida,4319000.00,NaN,2.0,2.0,23.2747317,-106.4592768,easybroker,2024-09-02 12:14:20.690010,mazatlan,NaN
4,mangle marina golf residences,2890000.00,NaN,2.0,2.0,23.2786341,-106.4536404,easybroker,2024-09-02 12:14:20.690010,mazatlan,NaN
5,torre playa azul,2800000.00,NaN,2.0,2.0,23.2028359,-106.4165282,easybroker,2024-09-02 12:14:20.690010,mazatlan,NaN
6,casa en venta en mediterraneo,5800000.00,169.50,3.0,3.0,23.2693327,-106.4445612,easybroker,2024-09-02 12:14:20.690010,mazatlan,34218.289086
7,condominio en venta en las gavias grand,14848500.00,160.71,3.0,3.0,23.2828623755,-106.4709091479,easybroker,2024-09-02 12:14:20.690010,mazatlan,92393.130483
8,cristal bay ... uno de los penthouses mas lujo...,24900000.00,450.00,NaN,3.0,23.2273616,-106.4288318,easybroker,2024-09-02 12:14:20.690010,mazatlan,55333.333333
9,cristal bay ... uno de los departamentos mas ...,8170000.00,125.00,NaN,2.0,23.231974,-106.4347995,easybroker,2024-09-02 12:14:20.690010,mazatlan,65360.000000
